In [19]:
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import pickle
import requests
import datetime
import re

pd.set_option("display.max_columns", 250)
pd.set_option("display.max_rows", 73)

In [8]:
team_abbrevs = ['crd', 'atl', 'rav', 'buf', 'car', 'chi', 'cin', 'cle', 'dal', 'den',
                'det', 'gnb', 'htx', 'clt', 'jax', 'kan', 'mia', 'min', 'nwe', 'nor',
                'nyg', 'nyj', 'rai', 'phi', 'pit', 'sdg', 'sfo', 'sea', 'ram', 'tam',
                'oti', 'was']

In [63]:
url = "https://www.pro-football-reference.com/teams/chi/2019/gamelog/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [64]:
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-root="/home/pfr/deploy/www" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
 <head>
  <!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
  <script async="true" type="text/javascript">
   (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '__tcfapiLocator';
	    var queue = [];
	    var win = window;
	    var cmpFrame;
	    
	    function addFrame() {
		var doc = win.document;
		var otherCMP = !!(win.frames[TCF_LOCATOR_NAME]);
		
		if (!oth

In [80]:
table = soup.find('table')
rows = table.find_all('tr')
i = 1
for row in rows[2:]:
    print(i)
    print(row, '\n')
    i += 1

1
<tr id="gamelog2019.1"><th class="right" csk="1" data-stat="week_num" scope="row">1</th><td class="left" data-stat="game_day_of_week">Thu</td><td class="left" csk="2019-09-05" data-stat="game_date">September 5</td><td class="center" data-stat="boxscore_word"><a href="/boxscores/201909050chi.htm">boxscore</a></td><td class="right" data-stat="game_outcome">L</td><td class="left iz" data-stat="overtime"></td><td class="right iz" data-stat="game_location"></td><td class="left" data-stat="opp"><a href="/teams/gnb/2019.htm">Green Bay Packers</a></td><td class="right" data-stat="pts_off">3</td><td class="right" data-stat="pts_def">10</td><td class="right" data-stat="pass_cmp">26</td><td class="right" data-stat="pass_att">45</td><td class="right" data-stat="pass_yds">208</td><td class="right iz" data-stat="pass_td">0</td><td class="right" data-stat="pass_int">1</td><td class="right" data-stat="pass_sacked">5</td><td class="right" data-stat="pass_sacked_yds">20</td><td class="right" data-stat

In [114]:
year = 2019

games = []
for team in team_abbrevs:
    url = f"https://www.pro-football-reference.com/teams/{team}/{year}/gamelog/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    try:
        table = soup.find('table')
        rows = table.find_all('tr')
    except:
        continue
    
    # get regular season games
    for row in rows[2:]:
        game_stats = {}
        # pull week_num
        week_num = re.search(r"data-stat=\"week_num\" scope=\"row\">([0-9]+)", str(row))[1]
        game_stats['week_num'] = float(week_num)
        
        game_stats['team'] = team
        opponent = re.search(r"data-stat=\"opp\"><a href=\"/teams/([\w]{3})/[0-9]{4}\.htm\">[ \w]+</a>", str(row))[1]
        game_stats['opp'] = opponent

        # pull all stats
        results = re.findall(r"data-stat=\"([\w]+)\">([ :@\w-]+)", str(row))
        for stat, val in results:
            if val.isdigit():
                game_stats[stat] = float(val)
            else:
                game_stats[stat] = str(val)
        games.append(game_stats)
    #     print(row.prettify(), '\n')

In [117]:
df = pd.DataFrame(games)
df.head()

,week_num,team,opp,game_day_of_week,game_date,game_outcome,overtime,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,third_down_success,third_down_att,fourth_down_success,fourth_down_att,time_of_poss,game_location
0,1.0,crd,det,Sun,September 8,T,OT,27.0,27.0,29.0,54.0,275.0,2.0,1.0,5.0,33.0,5.0,4.0,53.0,72.0,23.0,112.0,4.0,0.0,4.0,4.0,1.0,1.0,8.0,380.0,9.0,22.0,0.0,0.0,30:37,NaN
1,2.0,crd,rav,Sun,September 15,L,NaN,17.0,23.0,25.0,40.0,329.0,0.0,0.0,3.0,20.0,8.0,7.0,62.0,88.0,11.0,20.0,1.0,1.0,3.0,3.0,0.0,0.0,5.0,267.0,2.0,11.0,1.0,1.0,22:22,@
2,3.0,crd,car,Sun,September 22,L,NaN,20.0,38.0,30.0,43.0,127.0,2.0,2.0,8.0,46.0,4.0,2.0,69.0,68.0,22.0,121.0,5.0,0.0,2.0,2.0,2.0,2.0,5.0,212.0,9.0,17.0,1.0,1.0,31:44,NaN
3,4.0,crd,sea,Sun,September 29,L,NaN,10.0,27.0,22.0,32.0,206.0,0.0,1.0,4.0,35.0,7.0,5.0,68.0,73.0,23.0,115.0,5.0,1.0,1.0,3.0,1.0,1.0,3.0,141.0,3.0,9.0,0.0,0.0,26:36,NaN
4,5.0,crd,cin,Sun,October 6,W,NaN,26.0,23.0,20.0,32.0,248.0,0.0,0.0,1.0,5.0,7.0,7.0,62.0,86.0,38.0,266.0,7.0,2.0,4.0,5.0,2.0,2.0,3.0,150.0,6.0,13.0,1.0,1.0,31:03,@


In [118]:
df.shape

(512, 36)

In [152]:
# playoff team url
url_playoff = "https://www.pro-football-reference.com/teams/nwe/2019/gamelog/"
url_no_playoff = "https://www.pro-football-reference.com/teams/det/2019/gamelog/"

In [153]:
response = requests.get(url_playoff)
soup_playoff = BeautifulSoup(response.text, 'lxml')

response = requests.get(url_no_playoff)
soup_no_playoff = BeautifulSoup(response.text, 'lxml')

In [154]:
p_num_tables = len(soup_playoff.find_all('table'))
np_num_tables = len(soup_no_playoff.find_all('table'))

In [155]:
p_num_tables, np_num_tables

(4, 2)

In [135]:
table = soup_playoff.find_all('table')[1]
rows = table.find_all('tr')
i = 1
for row in rows[2:]:
#     print(i)
#     print(row, '\n')
    result = re.search(r"gamelog_opp", str(row))
    print(result == None)
    i += 1

True
True
True


In [130]:

table = soup_no_playoff.find_all('table')[1]
rows = table.find_all('tr')
i = 1
for row in rows[2:]:
#     print(i)
#     print(row, '\n')
    result = re.search(r"gamelog_opp", str(row))
    print(result[0] == None)
    i += 1

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [162]:
# playoff games
table = soup_playoff.find_all('table')[1]
rows = table.find_all('tr')
i = 1
for row in rows[2:]:
    print(i)
    print(row, '\n')
    i += 1

1
<tr id="playoff_gamelog2019.18"><th class="right" csk="18" data-stat="week_num" scope="row">18</th><td class="left" data-stat="game_day_of_week">Sat</td><td class="left" csk="2020-01-04" data-stat="game_date">January 4</td><td class="center" data-stat="boxscore_word"><a href="/boxscores/202001040nwe.htm">boxscore</a></td><td class="right" data-stat="game_outcome">L</td><td class="left iz" data-stat="overtime"></td><td class="right iz" data-stat="game_location"></td><td class="left" data-stat="opp"><a href="/teams/oti/2019.htm">Tennessee Titans</a></td><td class="right" data-stat="pts_off">13</td><td class="right" data-stat="pts_def">20</td><td class="right" data-stat="pass_cmp">20</td><td class="right" data-stat="pass_att">37</td><td class="right" data-stat="pass_yds">209</td><td class="right iz" data-stat="pass_td">0</td><td class="right" data-stat="pass_int">1</td><td class="right iz" data-stat="pass_sacked">0</td><td class="right iz" data-stat="pass_sacked_yds">0</td><td class="ri

In [164]:

year = 2019

playoff_games = []
for team in team_abbrevs:
    url = f"https://www.pro-football-reference.com/teams/{team}/{year}/gamelog/"
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    try:
        tables = soup.find_all('table')
    except:
        continue
    if len(tables) == 2:
        continue
    
    table = soup.find_all('table')[1]
    rows = table.find_all('tr')

    
    # get regular season games
    for row in rows[2:]:
        game_stats = {}
        # pull week_num
        week_num = re.search(r"data-stat=\"week_num\" scope=\"row\">([0-9]+)", str(row))[1]
        game_stats['week_num'] = float(week_num)
        
        game_stats['week_num']
        
        game_stats['team'] = team
        opponent = re.search(r"data-stat=\"opp\"><a href=\"/teams/([\w]{3})/[0-9]{4}\.htm\">[ \w]+</a>", str(row))[1]
        game_stats['opp'] = opponent

        # pull all stats
        results = re.findall(r"data-stat=\"([\w]+)\">([ :@\w-]+)", str(row))
        for stat, val in results:
            if val.isdigit():
                game_stats[stat] = float(val)
            else:
                game_stats[stat] = str(val)
        playoff_games.append(game_stats)
    #     print(row.prettify(), '\n')

In [165]:
playoff_df = pd.DataFrame(playoff_games)
playoff_df

,week_num,team,opp,game_day_of_week,game_date,game_outcome,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,third_down_success,third_down_att,fourth_down_success,fourth_down_att,time_of_poss,overtime,game_location
0,19.0,rav,oti,Sat,January 11,L,12.0,28.0,31.0,59.0,345.0,1.0,2.0,4.0,20.0,6.0,5.0,52.0,61.0,29.0,185.0,6.0,0.0,2.0,2.0,0.0,0.0,1.0,44.0,11.0,18.0,0.0,4.0,32:06,NaN,NaN
1,18.0,buf,htx,Sat,January 4,L,19.0,22.0,25.0,48.0,253.0,1.0,0.0,3.0,27.0,5.0,5.0,52.0,74.0,30.0,172.0,5.0,0.0,4.0,4.0,1.0,1.0,4.0,179.0,11.0,21.0,0.0,1.0,35:15,OT,@
2,19.0,gnb,sea,Sun,January 12,W,28.0,23.0,16.0,27.0,235.0,2.0,0.0,2.0,8.0,9.0,8.0,59.0,112.0,30.0,109.0,3.0,2.0,0.0,0.0,4.0,4.0,4.0,173.0,9.0,14.0,0.0,0.0,29:29,NaN,NaN
3,20.0,gnb,sfo,Sun,January 19,L,20.0,37.0,31.0,39.0,296.0,2.0,2.0,3.0,30.0,8.0,7.0,79.0,94.0,16.0,62.0,3.0,1.0,0.0,0.0,2.0,2.0,4.0,172.0,3.0,9.0,2.0,2.0,31:13,NaN,@
4,18.0,htx,buf,Sat,January 4,W,22.0,19.0,20.0,25.0,219.0,1.0,0.0,7.0,28.0,9.0,6.0,80.0,116.0,33.0,141.0,4.0,1.0,2.0,2.0,0.0,0.0,5.0,216.0,6.0,13.0,0.0,1.0,36:25,OT,NaN
5,19.0,htx,kan,Sun,January 12,L,31.0,51.0,31.0,52.0,348.0,2.0,0.0,5.0,40.0,7.0,6.0,59.0,92.0,21.0,94.0,4.0,1.0,1.0,2.0,4.0,4.0,3.0,137.0,5.0,15.0,1.0,5.0,34:35,NaN,@
6,19.0,kan,htx,Sun,January 12,W,51.0,31.0,23.0,35.0,316.0,5.0,0.0,1.0,5.0,9.0,8.0,65.0,134.0,21.0,118.0,5.0,2.0,1.0,1.0,6.0,7.0,4.0,123.0,2.0,8.0,0.0,0.0,25:25,NaN,NaN
7,20.0,kan,oti,Sun,January 19,W,35.0,24.0,23.0,35.0,292.0,3.0,0.0,2.0,2.0,8.0,7.0,65.0,120.0,27.0,112.0,4.0,2.0,0.0,0.0,5.0,5.0,3.0,122.0,6.0,10.0,1.0,1.0,29:50,NaN,NaN
8,21.0,kan,sfo,Sun,February 2,W,31.0,20.0,26.0,42.0,268.0,2.0,2.0,4.0,18.0,6.0,5.0,61.0,76.0,29.0,129.0,4.0,2.0,1.0,1.0,4.0,4.0,2.0,100.0,6.0,14.0,2.0,3.0,33:13,NaN,N
9,18.0,min,nor,Sun,January 5,W,26.0,20.0,19.0,31.0,226.0,1.0,0.0,3.0,16.0,7.0,6.0,61.0,94.0,40.0,136.0,3.0,2.0,2.0,2.0,2.0,2.0,6.0,298.0,10.0,18.0,0.0,0.0,36:56,OT,@


In [161]:
playoff_df.shape

(22, 36)

In [ ]:
def pull_stats(table):
    rows = table.find_all('tr')
    for row in rows[2:]:
        game_stats = {}
        # add year 
        game_stats['season_year'] = year
        # pull week_num
        week_num = re.search(r"data-stat=\"week_num\" scope=\"row\">([0-9]+)", str(row))[1]
        game_stats['week_num'] = float(week_num)

        game_stats['team'] = team
        opponent = re.search(r"data-stat=\"opp\"><a href=\"/teams/([\w]{3})/[0-9]{4}\.htm\">[ \w]+</a>", str(row))[1]
        game_stats['opp'] = opponent

        # pull all stats
        results = re.findall(r"data-stat=\"([\w]+)\">([ :@\w-]+)", str(row))
        for stat, val in results:
            if val.isdigit():
                game_stats[stat] = float(val)
            else:
                game_stats[stat] = str(val)
        games.append(game_stats)
        

In [178]:
# combined playoff + reg season
start_year = 2019
end_year = 2020

games = []
for year in range(start_year, end_year + 1):

    for team in team_abbrevs:
        url = f"https://www.pro-football-reference.com/teams/{team}/{year}/gamelog/"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        tables = soup.find_all('table')
        
        num_tables = len(tables)
        if num_tables == 2:
            playoff_team = False
        else:
            playoff_team = True

        # get regular season stats for all team years, playoffs only for playoff teams

        tables = soup.find_all('table')
        reg_table = tables[0]
            
        tables_to_parse = [reg_table]
        
        if playoff_team:
            playoff_table = tables[1]
            tables_to_parse.append(playoff_table)
        
        for table in tables_to_parse:
            # get regular season games
            rows = table.find_all('tr')
            for row in rows[2:]:
                game_stats = {}
                # add year 
                game_stats['season_year'] = year
                # pull week_num
                week_num = re.search(r"data-stat=\"week_num\" scope=\"row\">([0-9]+)", str(row))[1]
                game_stats['week_num'] = float(week_num)
                
                full_game_date = \
                    re.search(r"class=\"left\" csk=\"([-0-9]+)\" data-stat=\"game_date", str(row))[1]
                full_game_date = datetime.datetime.strptime(full_game_date, "%Y-%m-%d")
                game_stats['full_game_date'] = full_game_date

                game_stats['team'] = team
                opponent = re.search(r"data-stat=\"opp\"><a href=\"/teams/([\w]{3})/[0-9]{4}\.htm\">[ \w]+</a>", str(row))[1]
                game_stats['opp'] = opponent

                # pull all stats
                results = re.findall(r"data-stat=\"([\w]+)\">([ :@\w-]+)", str(row))
                for stat, val in results:
                    if val.isdigit():
                        game_stats[stat] = float(val)
                    else:
                        game_stats[stat] = str(val)
                games.append(game_stats)


In [179]:
g_df = pd.DataFrame(games)
g_df.sample()

,season_year,week_num,full_game_date,team,opp,game_day_of_week,game_date,game_outcome,overtime,pts_off,pts_def,pass_cmp,pass_att,pass_yds,pass_td,pass_int,pass_sacked,pass_sacked_yds,pass_yds_per_att,pass_net_yds_per_att,pass_cmp_perc,pass_rating,rush_att,rush_yds,rush_yds_per_att,rush_td,fgm,fga,xpm,xpa,punt,punt_yds,third_down_success,third_down_att,fourth_down_success,fourth_down_att,time_of_poss,game_location
54,2019,7.0,2019-10-20,buf,mia,Sun,October 20,W,NaN,31.0,21.0,16.0,26.0,188.0,2.0,0.0,2.0,14.0,7.0,6.0,61.0,109.0,23.0,117.0,5.0,0.0,3.0,3.0,2.0,2.0,3.0,127.0,3.0,10.0,0.0,0.0,26:29,NaN


In [168]:
import datetime

In [169]:
d = "2019-09-08"

In [171]:
datetime.datetime.strptime(d, "%Y-%m-%d")

datetime.datetime(2019, 9, 8, 0, 0)

In [47]:
len(soup.find_all('tbody'))

3

In [59]:
games_table = soup.find_all('tbody')[2]

In [60]:
for row in games_table:
    print(row)



<tr><th class="left" data-stat="player" scope="row">Team Stats</th><td class="right" data-stat="third_down_att">205</td><td class="right" data-stat="third_down_success">71</td><td class="right" data-stat="third_down_pct">34.6</td><td class="right" data-stat="fourth_down_att">28</td><td class="right" data-stat="fourth_down_success">17</td><td class="right" data-stat="fourth_down_pct">60.7</td><td class="right" data-stat="red_zone_att">55</td><td class="right" data-stat="red_zone_scores">31</td><td class="right" data-stat="red_zone_pct">56.4</td></tr>


<tr><th class="left" data-stat="player" scope="row">Opp. Stats</th><td class="right" data-stat="third_down_att">210</td><td class="right" data-stat="third_down_success">80</td><td class="right" data-stat="third_down_pct">38.1</td><td class="right" data-stat="fourth_down_att">22</td><td class="right" data-stat="fourth_down_success">13</td><td class="right" data-stat="fourth_down_pct">59.1</td><td class="right" data-stat="red_zone_att">53

In [48]:
tables = soup.find_all('tbody')
sched_table = tables[1].find_all('th')
for row in sched_table:
    print(row.prettify())

<th class="right" csk="1" data-stat="week_num" scope="row">
 1
</th>

<th class="right" csk="2" data-stat="week_num" scope="row">
 2
</th>

<th class="right" csk="3" data-stat="week_num" scope="row">
 3
</th>

<th class="right" csk="4" data-stat="week_num" scope="row">
 4
</th>

<th class="right" csk="5" data-stat="week_num" scope="row">
 5
</th>

<th class="right" csk="6" data-stat="week_num" scope="row">
 6
</th>

<th class="right" csk="7" data-stat="week_num" scope="row">
 7
</th>

<th class="right" csk="8" data-stat="week_num" scope="row">
 8
</th>

<th class="right" csk="9" data-stat="week_num" scope="row">
 9
</th>

<th class="right" csk="10" data-stat="week_num" scope="row">
 10
</th>

<th class="right" csk="10.8" data-stat="week_num" scope="row">
 11
</th>

<th class="right" csk="12" data-stat="week_num" scope="row">
 12
</th>

<th class="right" csk="13" data-stat="week_num" scope="row">
 13
</th>

<th class="right" csk="14" data-stat="week_num" scope="row">
 14
</th>

<th clas

In [49]:
tables = soup.find_all('tbody')
sched_table = tables[1].find_all('th')
sched_table = tables[1].find_all('tr')

for row in tables[1]:
    print(row, '\n')


 

<tr><th class="right" csk="1" data-stat="week_num" scope="row">1</th><td class="left" data-stat="game_day_of_week">Sun</td><td class="left" csk="2020-09-13" data-stat="game_date">September 13</td><td class="right" csk="13.00" data-stat="game_time">1:00PM ET</td><td class="center" data-stat="boxscore_word"><a href="/boxscores/202009130det.htm">boxscore</a></td><td class="right" data-stat="game_outcome">W</td><td class="left iz" data-stat="overtime"></td><td class="center" data-stat="team_record">1-0</td><td class="right" data-stat="game_location">@</td><td class="left" data-stat="opp"><a href="/teams/det/2020.htm">Detroit Lions</a></td><td class="right" data-stat="pts_off">27</td><td class="right" data-stat="pts_def">23</td><td class="right" data-stat="first_down_off">26</td><td class="right" data-stat="yards_off">363</td><td class="right" data-stat="pass_yds_off">214</td><td class="right" data-stat="rush_yds_off">149</td><td class="right iz" data-stat="to_off"></td><td class="right

In [50]:
games = []
tables = soup.find_all('tbody')

for row in tables[1]:
    game = {}
    
    # add 'opp' column
    try: 
        opponent = re.search(r"href=\"/teams/([\w]{3}).*", str(row))[1]
        game['opp'] = opponent
    except:
        pass
    
    # add week_num column
    try:
        week_num = re.search(r"\"week_num\" scope=\"row\">([0-9]+)", str(row))[1]
        game['week_num'] = week_num
    except:
        pass
#     add stat columns        
#     print(opponent)
    results = re.findall(r"data-stat=\"([\w]+)\">([@\w-]+)", str(row))
    
    # some info is stored in the 'th' tags
#     print(results)

    for result in results:
#         print(result)
        if result != []:
            stat, val = result
#             print(stat, '\n', val)
            try:
                game[stat] = float(val)
            except:
#                 print(stat)
                game[stat] = val
    if len(game) > 0:
        games.append(game)
    else:
        continue

In [51]:
pd.DataFrame(games)

,opp,week_num,game_day_of_week,game_date,game_time,game_outcome,team_record,game_location,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st,to_off,overtime
0,det,1,Sun,September,1.0,W,1-0,@,27.0,23.0,26.0,363.0,214.0,149.0,25.0,426.0,288.0,138.0,1.0,10.0,-7.0,0.0,NaN,NaN
1,nyg,2,Sun,September,1.0,W,2-0,NaN,17.0,13.0,18.0,304.0,169.0,135.0,22.0,295.0,220.0,75.0,2.0,3.0,-0.0,-3.0,2.0,NaN
2,atl,3,Sun,September,1.0,W,3-0,@,30.0,26.0,24.0,437.0,307.0,130.0,22.0,371.0,227.0,144.0,1.0,2.0,-5.0,5.0,2.0,NaN
3,clt,4,Sun,October,1.0,L,3-1,NaN,11.0,19.0,16.0,269.0,241.0,28.0,18.0,289.0,186.0,103.0,NaN,-1.0,-2.0,-7.0,1.0,NaN
4,tam,5,Thu,October,8.0,W,4-1,NaN,20.0,19.0,18.0,243.0,208.0,35.0,18.0,339.0,233.0,106.0,1.0,-0.0,3.0,1.0,1.0,NaN
5,car,6,Sun,October,1.0,W,5-1,@,23.0,16.0,22.0,261.0,198.0,63.0,23.0,303.0,191.0,112.0,3.0,1.0,3.0,2.0,1.0,NaN
6,ram,7,Mon,October,8.0,L,5-2,@,10.0,24.0,14.0,279.0,230.0,49.0,24.0,371.0,211.0,160.0,1.0,-17.0,-3.0,5.0,2.0,NaN
7,nor,8,Sun,November,4.0,L,5-3,NaN,23.0,26.0,18.0,329.0,233.0,96.0,23.0,394.0,272.0,122.0,NaN,0.0,-6.0,4.0,1.0,OT
8,oti,9,Sun,November,1.0,L,5-4,@,17.0,24.0,22.0,375.0,319.0,56.0,11.0,228.0,136.0,92.0,NaN,-12.0,3.0,1.0,2.0,NaN
9,min,10,Mon,November,8.0,L,5-5,NaN,13.0,19.0,10.0,149.0,108.0,41.0,19.0,385.0,286.0,99.0,2.0,-18.0,3.0,8.0,2.0,NaN


In [ ]:
games

In [23]:
games = []
start_year = 2019
end_year = 2020

for team in team_abbrevs:
    
    for year in range(start_year, end_year + 1):
        
        url = f"https://www.pro-football-reference.com/teams/{team}/{year}.htm"

        response = requests.get(url)
        if response.status_code == 404:
            continue

        soup = BeautifulSoup(response.text, "lxml")
        tables = soup.find_all('tbody')
        sched_table = tables[1].find_all('tr')
        
        for row in sched_table:
            game = {}
            game["team"] = team
            game["year"] = year

            try: 
                opponent = re.search(r"href=\"/teams/([\w]{3}).*", str(row))[1]
                game['opp'] = opponent
            except:
                pass
            
            # get week num (syntax is a little different)
            try:
                week_num = re.search(r"\"week_num\" scope=\"row\">([\w ]+)", str(row))[1]
                game['week_num'] = week_num
                
            except:
                pass
            
        #     print(opponent)
            results = re.findall(r"data-stat=\"([\w]+)\">([\w-]+)", str(row))
            for result in results:
        #         print(result)
                if result != []:
                    stat, val = result
                    try:
                        game[stat] = float(val)
                    except:
                        game[stat] = val
            if len(game) > 0:
                games.append(game)
            else:
                continue
            
        

In [24]:
game_df = pd.DataFrame(games)

game_df.sample(20)

,team,year,opp,week_num,game_day_of_week,game_date,game_time,game_outcome,overtime,team_record,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st,game_location
502,clt,2020,buf,Wild Card,Sat,January,1.0,L,NaN,11-6,24.0,27.0,27.0,472.0,309.0,163.0,NaN,22.0,397.0,301.0,96.0,NaN,18.0,-15.0,-4.0,NaN
247,cin,2020,dal,14,Sun,December,1.0,L,NaN,2-10-1,7.0,30.0,18.0,309.0,208.0,101.0,3.0,16.0,272.0,171.0,101.0,NaN,-14.0,-8.0,-0.0,NaN
278,cle,2020,phi,11,Sun,November,1.0,W,NaN,7-3,22.0,17.0,17.0,324.0,187.0,137.0,1.0,19.0,315.0,209.0,106.0,3.0,-4.0,10.0,1.0,NaN
1101,oti,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,rav,2019,nwe,9,Sun,November,8.0,W,NaN,6-2,37.0,20.0,26.0,372.0,162.0,210.0,2.0,22.0,342.0,268.0,74.0,2.0,14.0,5.0,-5.0,NaN
975,sea,2019,cle,6,Sun,October,1.0,W,NaN,5-1,32.0,28.0,29.0,454.0,284.0,170.0,1.0,21.0,406.0,249.0,157.0,4.0,15.0,-3.0,-4.0,NaN
1060,tam,2019,det,15,Sun,December,1.0,W,NaN,7-7,38.0,17.0,23.0,495.0,446.0,49.0,1.0,16.0,295.0,250.0,45.0,3.0,13.0,14.0,-7.0,NaN
508,jax,2019,nor,6,Sun,October,1.0,L,NaN,2-4,6.0,13.0,11.0,226.0,151.0,75.0,1.0,18.0,326.0,222.0,104.0,NaN,-11.0,-1.0,0.0,NaN
1152,was,2020,dal,12,Thu,November,4.0,W,NaN,4-7,41.0,16.0,22.0,338.0,156.0,182.0,1.0,13.0,247.0,187.0,60.0,2.0,8.0,17.0,1.0,NaN
937,sfo,2019,ram,6,Sun,October,4.0,W,NaN,5-0,20.0,7.0,22.0,331.0,232.0,99.0,2.0,10.0,157.0,48.0,109.0,1.0,-3.0,25.0,-5.0,NaN


Get touchdowns

In [ ]:
def scrape_games(start_year=1980, end_year=2000, save_to=False):
    """
    Builds a DataFrame containing all games from start_year to end_year (inclusive).


    """

    games = []

    for team in team_abbrevs:

        for year in range(start_year, end_year + 1):

            url = f"https://www.pro-football-reference.com/teams/{team}/{year}.htm"

            response = requests.get(url)
            if response.status_code == 404:
                continue

            soup = BeautifulSoup(response.text, "lxml")
            tables = soup.find_all("tbody")
            sched_table = tables[1].find_all("tr")

            for row in sched_table:
                game = {}
                game["team"] = team
                game["year"] = year

                try:
                    opponent = re.search(r"href=\"/teams/([\w]{3}).*", str(row))[1]
                    game["opp"] = opponent
                except:
                    pass
                # (try) to get week number of game
                try:
                    week_num = re.search(
                        r"\"week_num\" scope=\"row\">([\w ]+)", str(row)
                    )[1]
                    game["week_num"] = week_num
                except:
                    pass
                #     print(opponent)
                results = re.findall(r"data-stat=\"([\w]+)\">([@ \w-]+)", str(row))
                for result in results:
                    #         print(result)
                    if result != []:
                        stat, val = result
                        try:
                            game[stat] = float(val)
                        except:
                            game[stat] = val
                if len(game) > 0:
                    games.append(game)
                else:
                    continue

    game_df = pd.DataFrame(games)

    if save_to:
        game_df.to_pickle("data/" + str(save_to))

    return game_df


In [38]:
url = f"https://www.pro-football-reference.com/teams/chi/2019.htm"

response = requests.get(url)


soup = BeautifulSoup(response.text, "lxml")
tables = soup.find_all("tbody")
sched_table = tables[1].find_all("tr")
for row in sched_table:
    print(row.prettify(), '\n\n')

<tr>
 <th class="right" csk="1" data-stat="week_num" scope="row">
  1
 </th>
 <td class="left" data-stat="game_day_of_week">
  Thu
 </td>
 <td class="left" csk="2019-09-05" data-stat="game_date">
  September 5
 </td>
 <td class="right" csk="20.20" data-stat="game_time">
  8:20PM ET
 </td>
 <td class="center" data-stat="boxscore_word">
  <a href="/boxscores/201909050chi.htm">
   boxscore
  </a>
 </td>
 <td class="right" data-stat="game_outcome">
  L
 </td>
 <td class="left iz" data-stat="overtime">
 </td>
 <td class="center" data-stat="team_record">
  0-1
 </td>
 <td class="right iz" data-stat="game_location">
 </td>
 <td class="left" data-stat="opp">
  <a href="/teams/gnb/2019.htm">
   Green Bay Packers
  </a>
 </td>
 <td class="right" data-stat="pts_off">
  3
 </td>
 <td class="right" data-stat="pts_def">
  10
 </td>
 <td class="right" data-stat="first_down_off">
  16
 </td>
 <td class="right" data-stat="yards_off">
  254
 </td>
 <td class="right" data-stat="pass_yds_off">
  208
 </td